# Business Understanding 
- Reuniões com stakeholders, definição dos objetivos, restrições, e KPIs (Key Performance Indicators).
## Context 

## Goals

# Data Understanding

- Coleta de dados, análise exploratória, identificação de padrões e outliers

## Libraries import and loading the data

- Coleta de dados, análise exploratória, identificação de padrões e outliers.

## Data quality 

## Exploratory Data Analysis

# Data Preparation

-  Limpeza de dados, criação de variáveis derivadas, normalização, transformação de dados categóricos em numéricos.

## Data manipulation

## Pre-processing

# Modeling
- Seleção de algoritmos, divisão dos dados em conjuntos de treino e teste, treinamento e avaliação de modelos.
## Base model

## Optimized model

# Evaluation
## Metrics

## Graphics

# Conclusions